# KinSight
A family-focused neural network to identify my household people and paws.

This is am image classifier based on pre-trained CNN ResNet.

# 1. Dataset Format

family_dataset/
│
├── train/
│   ├── Marvin/
│   │   ├── img001.jpg
│   │   ├── img002.jpg
│   │   └── ...
│   ├── Jane/
│   ├── Eric/
│   ├── Tobin/
│   └── Wesley/
│
└── test/
    ├── Marvin/
    ├── Jane/
    ├── Eric/
    ├── Tobin/
    └── Wesley/

# Choosing this format is because:
- PyTorch’s ImageFolder class automatically reads class names from the subfolder names.
- It returns each image as a tensor and its class label as an integer.
- It avoids custom binary formats like IDX — your dataset stays human-readable and easy to extend.


In [1]:
# 2. Loading your dataset in PyTorch

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(root='family_dataset/train', transform=transform)
test_dataset  = datasets.ImageFolder(root='family_dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)

FileNotFoundError: Found no valid file for the classes Eric, Jane, Tobin, Welsey. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp